# VIRTUAL ASSISTANT 🤖

# Load Dependencies 👩‍💻👨‍💻

❗❗ GENERAR CUSTOM DATA SET con alumnos



Antes de ejecutar:
- Cargar una carpeta docs/ con archivos .txt que serán la knowledge base
- Usar T4 en el ambiente (Entorno de Ejecución -> Cambiar tipo de ambiente -> T4 GPU) o modificar 'cuda' a 'gpu'
- Cargar un archivo .env con las API KEYS de HuggingFace/OpenAI y Pinecone

In [ ]:
pip install openai langchain langchain-community pinecone-client python-dotenv langchain-pinecone tiktoken sentence_transformers chromadb  langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 12.5 MB/s eta

In [ ]:
pip install --upgrade langchain langchain-community



SyntaxError: invalid syntax (ipython-input-3392072582.py, line 1)

In [ ]:
import os
import logging

from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint, HuggingFacePipeline
from langchain_openai import ChatOpenAI
import  sys, warnings
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings

logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] - %(message)s',
    handlers=[
        logging.FileHandler('/content/langchaindemo.log', mode='w'),
        logging.StreamHandler(),
    ],
    force=True
)

logger = logging.getLogger(__name__)
logger.info("LangChain Demo Initialized")

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
warnings.filterwarnings("ignore", category=UserWarning)

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
[INFO] - LangChain Demo Initialized


# RAG 📚✂️🧐🗃️📝

In [ ]:
# ------------- Retrieval-Augmented Generation  ------------- #

def get_docs():
    """
    Loads each file into one document, like knowledge base
    :return: docs
    """

    loader = DirectoryLoader("docs", "*.txt", loader_cls=TextLoader)  # Reads custom data from local files

    docs = loader.load()
    logging.info(f"Loaded {len(docs)} documents")

    return docs

def split_text(docs):
    """
    Get chunks from docs. Our loaded doc may be too long for most models, and even if it fits is can struggle to find relevant context. So we generate chunks
    :param docs: docs to be split
    :return: chunks
    """

    text_splitter = RecursiveCharacterTextSplitter( # recommended splitter for generic text
        chunk_size=2000,
        chunk_overlap=200,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(docs)

    logging.info(f"Splitted into {len(chunks)} chunks")
    return chunks

def get_data_store(chunks):
    """
    Store chunks into a db. ChromaDB uses vector embeddings as the key, creates a new DB from the documents
    :param docs:
    :param chunks:
    :return: database
    """
    logging.info(f"Starting Embeddings and database setup")
    embeddings = HuggingFaceEmbeddings( #  embedding=OpenAIEmbeddings() rate limit
        model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device': 'cpu'} # TODO gpu
    )

    db = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    return db


# ------------------- Generate Response ------------------- #

def get_chain(db):
    """
    Generate a response using RAG with OpenAI Chat model
    """
    logging.info(f"Starting Retrieval Setup")
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",  # Model to use
        temperature=0.3,             # Controls randomness of output. 0 = deterministic, 1 = very creative. Lower values make answers more focused and precise.
        max_tokens=512               # Max tokens in the response.control cost and prevent long responses
    )

    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
        verbose=False
    )
    logging.info(f"Successful Setup")
    return chain


In [ ]:
docs = get_docs()           # Load custom files
chunks = split_text(docs)   # Split into chunks
db = get_data_store(chunks) # Generate vectorstore
chain = get_chain(db)       # Set LLM and context

[INFO] - Loaded 4 documents
[INFO] - Splitted into 4 chunks
[INFO] - Starting Embeddings and database setup
[INFO] - NumExpr defaulting to 2 threads.
[INFO] - TensorFlow version 2.19.0 available.
[INFO] - JAX version 0.5.3 available.
[INFO] - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
[INFO] - Starting Retrieval Setup
[INFO] - Successful Setup


In [ ]:
docs

[Document(metadata={'source': 'docs/temario.txt'}, page_content='Contenidos:\nUnidad 1: Introducción a Transformers\nIntroducción al concepto de Gen AI, LLMs y Transformers. Historia. Arquitectura. Mecanismo de Atención. Embeddings y Positional Encoding. Aplicaciones en la industria.\n\nUnidad 2: Algoritmos de Embedding y Positional Encoding\nAlgoritmos de Embedding y Positional Encoding. Transformer basando en N-grama\n\nUnidad 3: Fine Tuning\nReinforcement Learning. RLHF y sus security issues. Fine tuning. Pipeline productivo. \n\nUnidad 4: Responsible AI\nConsideraciones éticas en AI: biases en training data, fairness, impacto social, detección de contenido generado de forma artificial. Narrow AI vs. AGI. AGI como agente. Foundation models. Emergent capabilities. Security vulnerabilities. Interpretability. Alignment.\n\nUnidad 5: Retrieval Augmented Generation (RAG)\nIntroducción Retrieval Augmented Generation. Bases de datos vectoriales: Chroma DB y Pinecone, uso de embeddings y ef

# Generate Response ✏️

In [ ]:
prompt = "what is ai?"
response = chain.run(prompt)
response

/tmp/ipython-input-2295720742.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(prompt)


'AI stands for Artificial Intelligence. It refers to the simulation of human intelligence processes by machines, especially computer systems. These processes include learning, reasoning, problem-solving, perception, and language understanding. AI is used in various industries to automate tasks, improve efficiency, and make decisions based on data analysis.'

It answers even though there's no info about AI in my knowledge base (parametric memory)

# Create custom Chatbot 🤖

In [ ]:
print("Chatbot: Hola! ¿Qué deseas saber sobre 73.64 Temas Avanzandos de Deep Learning?")

user_input = ""

while user_input != "bye":
    user_input = input("You: ")
    response = chain.run(user_input + "Solo responde en español. Solo responde información con el contexto proporcinado. En caso de no tener información responde 'Lo siento solo tengo información sobre la materia temas avanzados en deep learning") # Get chatbot response to user prompt
    print(f"Chatbot: {response}")

Chatbot: Hola! ¿Qué deseas saber sobre 73.64 Temas Avanzandos de Deep Learning?
You: donde queda argentina?
Chatbot: Lo siento solo tengo información sobre la materia temas avanzados en deep learning.
You: bye
Chatbot: Lo siento solo tengo información sobre la materia temas avanzados en deep learning.
